# findling around with blocks

This is my test ground to get to know the Blocks framework for deep learning and make keep some basics examples.

In [8]:
#MNIST example of blocks : https://blocks.readthedocs.org/en/latest/tutorial.html
from theano import tensor
x = tensor.matrix('features')

from blocks.bricks import Linear, Rectifier, Softmax
input_to_hidden = Linear(name='input_to_hidden', input_dim=784, output_dim=100)
h = Rectifier().apply(input_to_hidden.apply(x))
hidden_to_output = Linear(name='hidden_to_output', input_dim=100, output_dim=10)
y_hat = Softmax().apply(hidden_to_output.apply(h))

y = tensor.lmatrix('targets')
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError
cost = CategoricalCrossEntropy().apply(y.flatten(), y_hat)


from blocks.bricks import WEIGHT
from blocks.graph import ComputationGraph
from blocks.filter import VariableFilter
cg = ComputationGraph(cost)
W1, W2 = VariableFilter(roles=[WEIGHT])(cg.variables)
cost = cost + 0.005 * (W1 ** 2).sum() + 0.005 * (W2 ** 2).sum()
cost.name = 'cost_with_regularization'


#Short for above
from blocks.bricks import MLP
x = tensor.matrix('features')
mlp = MLP(activations=[Rectifier(), Softmax()], dims=[784, 400, 10]).apply(x)

from blocks.initialization import IsotropicGaussian, Constant
input_to_hidden.weights_init = hidden_to_output.weights_init = IsotropicGaussian(0.01)
input_to_hidden.biases_init = hidden_to_output.biases_init = Constant(0)
input_to_hidden.initialize()
hidden_to_output.initialize()

W1.get_value() 

from fuel.datasets import MNIST
mnist = MNIST(("train",))


from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
data_stream = Flatten(DataStream.default_stream(
    mnist,
    iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=256)))

from blocks.algorithms import GradientDescent, Scale
algorithm = GradientDescent(cost=cost, parameters=cg.parameters,
                             step_rule=Scale(learning_rate=0.1))



mnist_test = MNIST(("test",))
data_stream_test = Flatten(DataStream.default_stream(
    mnist_test,
    iteration_scheme=SequentialScheme(
       mnist_test.num_examples, batch_size=64)))



from blocks.extensions.monitoring import DataStreamMonitoring,DataStreamMonitoring
classification_error= MisclassificationRate().apply(np.argmax(y_hat),y)
monitor = DataStreamMonitoring(
    variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
main_loop = MainLoop(data_stream=data_stream, algorithm=algorithm,
                     extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
main_loop.run() 


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 test_cost_with_regularization: 2.34214115531
	 test_misclassificationrate_apply_error_rate: 1.0


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: False
	 epochs_done: 1
	 iterations_done: 235
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 235:
	 test_cost_with_regulariz

In [24]:
#I started from the example above and added some couple of things


import numpy as np
#Importing dataset
from fuel.datasets import MNIST
mnist = MNIST(("train",))

#Data Stream
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
data_stream = Flatten(DataStream.default_stream(
    mnist,
    iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=256)))

#Model
from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor

x = tensor.matrix('features')
y1 = tensor.lmatrix(name='targets')

#mlp = MLP(activations=[Logistic(name='sigmoid_0'),
#          Rectifier(name='rect1'),Softmax(name='softmax_0')], dims=[784, 300, 200,10],
#          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))

mlp = MLP(activations=[Rectifier(name='rect0'),Logistic(name='sigmoid_1'),Softmax(name='softmax_0')], dims=[784, 300, 300, 10],
          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))


y = mlp.apply(x)
print(y)
mlp.push_initialization_config()
mlp.children[0].weights_init = Constant(0.01)
mlp.initialize()
#print(mlp.children[0].parameters[0].get_value()) 


#Cost
from blocks.graph import ComputationGraph
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError,MisclassificationRate
from blocks.algorithms import Scale,GradientDescent
cost = CategoricalCrossEntropy().apply(y1.flatten(),y)
algorithm = GradientDescent(cost=cost, parameters=ComputationGraph(cost).parameters,
                             step_rule=Scale(learning_rate=0.005))


#data stream test
mnist_test = MNIST(("test",))
data_stream_test = Flatten(DataStream.default_stream(
    mnist_test,
    iteration_scheme=SequentialScheme(
       mnist_test.num_examples, batch_size=64)))


#Monitor
from blocks.extensions.monitoring import DataStreamMonitoring
classification_error= MisclassificationRate().apply(np.argmax(y1),y)
monitor = DataStreamMonitoring(
    variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

#MainLoop
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
main_loop = MainLoop(algorithm,data_stream=data_stream, model=mlp,
                     extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
main_loop.run()

#%debug

mlp_apply_output

-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 test_categoricalcrossentropy_apply_cost: 28.1514429187
	 test_misclassificationrate_apply_error_rate: 0.891719745223


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: False
	 epochs_done: 1
	 iterations_done: 235
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the ite

# Results

batch_size, activation, dims, cost_used, learning_rate, epoch,  cost_with_regularization, misclassificationrate

64,[Logistic(), Rectifier(),Softmax()],[784, 300, 200,10], CategoricalCrossEntropy, 0.05, 4, 0.165451308803, 0.927249203822

...








In [ ]:

from itertools import permutations,product


for m,j,h in product([Logistic, Rectifier],[Logistic, Rectifier],[Logistic, Rectifier]):
    print(m,j,x)
    activation = [m(name=str(type(m))+'0'),j(name=str(type(j))+'1'),h(name=str(type(h))+'2'),Softmax(name='softmax_4')]
    print(activation)
    mlp = MLP(activations=activation, dims=[784,500,300,100,10],
          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))

    try_model_on_mnist(mlp)
    break
    

In [ ]:
%debug
print(list(permutations([Logistic, Rectifier],r=3)))
print(Logistic)
print(list(permutations([1,2,3],3)))

> /home/ntak/.local/lib/python2.7/site-packages/blocks/bricks/__init__.py(739)_push_allocation_config()
    738         if not len(self.dims) - 1 == len(self.linear_transformations):
--> 739             raise ValueError
    740         for input_dim, output_dim, layer in \

ipdb> self.dims
[784, 300, 200, 10]
ipdb> self.linear_transformations
[<blocks.bricks.Linear object at 0x7fb189943dd0: name=linear_0>, <blocks.bricks.Linear object at 0x7fb18c716a90: name=linear_1>, <blocks.bricks.Linear object at 0x7fb189943e50: name=linear_2>, <blocks.bricks.Linear object at 0x7fb189943c90: name=linear_3>]


In [27]:
import numpy as np

#Importing dataset
from fuel.datasets import MNIST
mnist = MNIST(("train",))
mnist_test = MNIST(("test",))

from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten

from blocks.graph import ComputationGraph
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError,MisclassificationRate
from blocks.algorithms import Scale,GradientDescent

from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor
from blocks.extensions.monitoring import DataStreamMonitoring

from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing


def try_model_on_mnist(model,batch_size=64,nb_epoch=4):
    #Data Stream
    data_stream = Flatten(DataStream.default_stream(
        mnist,
        iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=batch_size)))

    #Theano variables initialization

    x = tensor.matrix('features')
    y1 = tensor.lmatrix(name='targets')
    y = model.apply(x)
    model.push_initialization_config()
    model.children[0].weights_init = Constant(0.01)
    model.initialize()

    
    #Cost
    cost = CategoricalCrossEntropy().apply(y1.flatten(),y)
    algorithm = GradientDescent(cost=cost, parameters=ComputationGraph(cost).parameters,
                                 step_rule=Scale(learning_rate=0.005))

    #data stream test
    data_stream_test = Flatten(DataStream.default_stream(
        mnist_test,
        iteration_scheme=SequentialScheme(
           mnist_test.num_examples, batch_size=batch_size)))
    
    #Monitor
    classification_error= MisclassificationRate().apply(np.argmax(y1),y)
    monitor = DataStreamMonitoring(
        variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

    #MainLoop
    main_loop = MainLoop(algorithm,data_stream=data_stream, model=mlp,
                         extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
    main_loop.run()


Examples from internet

In [ ]:
# Example to cast numpy datasets to h5py from http://fuel.readthedocs.org/en/latest/h5py_dataset.html
import numpy
numpy.save(
    'train_vector_features.npy',
    numpy.random.normal(size=(90, 10)).astype('float32'))
numpy.save(
    'test_vector_features.npy',
    numpy.random.normal(size=(10, 10)).astype('float32'))
numpy.save(
    'train_image_features.npy',
numpy.random.randint(2, size=(90, 3, 5, 5)).astype('uint8'))
numpy.save(
    'test_image_features.npy',
    numpy.random.randint(2, size=(10, 3, 5, 5)).astype('uint8'))
numpy.save(
    'train_targets.npy',
    numpy.random.randint(10, size=(90, 1)).astype('uint8'))
numpy.save(
    'test_targets.npy',
    numpy.random.randint(10, size=(10, 1)).astype('uint8'))


train_vector_features = numpy.load('train_vector_features.npy')
test_vector_features = numpy.load('test_vector_features.npy')
train_image_features = numpy.load('train_image_features.npy')
test_image_features = numpy.load('test_image_features.npy')
train_targets = numpy.load('train_targets.npy')
test_targets = numpy.load('test_targets.npy')

import h5py
f = h5py.File('dataset.hdf5', mode='w')
vector_features = f.create_dataset(
    'vector_features', (100, 10), dtype='float32')
image_features = f.create_dataset(
    'image_features', (100, 3, 5, 5), dtype='uint8')
targets = f.create_dataset(
    'targets', (100, 1), dtype='uint8')

vector_features[...] = numpy.vstack(
    [train_vector_features, test_vector_features])
image_features[...] = numpy.vstack(
    [train_image_features, test_image_features])
targets[...] = numpy.vstack([train_targets, test_targets])


vector_features.dims[0].label = 'batch'
vector_features.dims[1].label = 'feature'
image_features.dims[0].label = 'batch'
image_features.dims[1].label = 'channel'
image_features.dims[2].label = 'height'
image_features.dims[3].label = 'width'
targets.dims[0].label = 'batch'
targets.dims[1].label = 'index'

split_array = numpy.empty(
    6,
    dtype=numpy.dtype([
        ('split', 'a', 5),
        ('source', 'a', 15),
        ('start', numpy.int64, 1),
        ('stop', numpy.int64, 1),
        ('indices', h5py.special_dtype(ref=h5py.Reference)),
        ('available', numpy.bool, 1),
        ('comment', 'a', 1)]))
split_array[0:3]['split'] = 'train'.encode('utf8')
split_array[3:6]['split'] = 'test'.encode('utf8')
split_array[0:6:3]['source'] = 'vector_features'.encode('utf8')
split_array[1:6:3]['source'] = 'image_features'.encode('utf8')
split_array[2:6:3]['source'] = 'targets'.encode('utf8')
split_array[0:3]['start'] = 0
split_array[0:3]['stop'] = 90
split_array[3:6]['start'] = 90
split_array[3:6]['stop'] = 100
split_array[:]['indices'] = h5py.Reference()
split_array[:]['available'] = True
split_array[:]['comment'] = '.'.encode('utf8')
f.attrs['split'] = split_array

from fuel.datasets.hdf5 import H5PYDataset
split_dict = {
    'train': {'vector_features': (0, 90), 'image_features': (0, 90),
              'targets': (0, 90)},
    'test': {'vector_features': (90, 100), 'image_features': (90, 100),
             'targets': (90, 100)}}
f.attrs['split'] = H5PYDataset.create_split_array(split_dict)

f.flush()
f.close()

#Manipulations

train_set = H5PYDataset('dataset.hdf5', which_sets=('train',))
print(train_set.num_examples)

test_set = H5PYDataset('dataset.hdf5', which_sets=('test',))
print(test_set.num_examples)

train_set = H5PYDataset(
    'dataset.hdf5', which_sets=('train',), subset=slice(0, 80))
print(train_set.num_examples)
valid_set = H5PYDataset(
    'dataset.hdf5', which_sets=('train',), subset=slice(80, 90))
print(valid_set.num_examples)

print(train_set.provides_sources)
print(train_set.axis_labels['image_features'])
print(train_set.axis_labels['vector_features'])
print(train_set.axis_labels['targets'])


handle = train_set.open()
data = train_set.get_data(handle, slice(0, 10))
print((data[0].shape, data[1].shape, data[2].shape))
train_set.close(handle)



In [9]:
#lenet example
"""Convolutional network example.
Run the training for 50 epochs with
```
python __init__.py --num-epochs 50
```
It is going to reach around 0.8% error rate on the test set.
"""
import logging
import numpy
from argparse import ArgumentParser

from theano import tensor

from blocks.algorithms import GradientDescent, Scale
from blocks.bricks import (MLP, Rectifier, Initializable, FeedforwardSequence,
                           Softmax)
from blocks.bricks.conv import (ConvolutionalActivation, ConvolutionalSequence,
                                Flattener, MaxPooling)
from blocks.bricks.cost import CategoricalCrossEntropy, MisclassificationRate
from blocks.extensions import FinishAfter, Timing, Printing, ProgressBar
from blocks.extensions.monitoring import (DataStreamMonitoring,
                                          TrainingDataMonitoring)
from blocks.extensions.saveload import Checkpoint
from blocks.graph import ComputationGraph
from blocks.initialization import Constant, Uniform
from blocks.main_loop import MainLoop
from blocks.model import Model
from blocks.monitoring import aggregation
from fuel.datasets import MNIST
from fuel.schemes import ShuffledScheme
from fuel.streams import DataStream
from toolz.itertoolz import interleave


class LeNet(FeedforwardSequence, Initializable):
    """LeNet-like convolutional network.
    The class implements LeNet, which is a convolutional sequence with
    an MLP on top (several fully-connected layers). For details see
    [LeCun95]_.
    .. [LeCun95] LeCun, Yann, et al.
       *Comparison of learning algorithms for handwritten digit
       recognition.*,
       International conference on artificial neural networks. Vol. 60.
    Parameters
    ----------
    conv_activations : list of :class:`.Brick`
        Activations for convolutional network.
    num_channels : int
        Number of channels in the input image.
    image_shape : tuple
        Input image shape.
    filter_sizes : list of tuples
        Filter sizes of :class:`.blocks.conv.ConvolutionalLayer`.
    feature_maps : list
        Number of filters for each of convolutions.
    pooling_sizes : list of tuples
        Sizes of max pooling for each convolutional layer.
    top_mlp_activations : list of :class:`.blocks.bricks.Activation`
        List of activations for the top MLP.
    top_mlp_dims : list
        Numbers of hidden units and the output dimension of the top MLP.
    conv_step : tuples
        Step of convolution (similar for all layers).
    border_mode : str
        Border mode of convolution (similar for all layers).
    """
    def __init__(self, conv_activations, num_channels, image_shape,
                 filter_sizes, feature_maps, pooling_sizes,
                 top_mlp_activations, top_mlp_dims,
                 conv_step=None, border_mode='valid', **kwargs):
        if conv_step is None:
            self.conv_step = (1, 1)
        else:
            self.conv_step = conv_step
        self.num_channels = num_channels
        self.image_shape = image_shape
        self.top_mlp_activations = top_mlp_activations
        self.top_mlp_dims = top_mlp_dims
        self.border_mode = border_mode

        conv_parameters = zip(conv_activations, filter_sizes, feature_maps)

        # Construct convolutional layers with corresponding parameters
        self.layers = list(interleave([
            (ConvolutionalActivation(filter_size=filter_size,
                                     num_filters=num_filter,
                                     activation=activation.apply,
                                     step=self.conv_step,
                                     border_mode=self.border_mode,
                                     name='conv_{}'.format(i))
             for i, (activation, filter_size, num_filter)
             in enumerate(conv_parameters)),
            (MaxPooling(size, name='pool_{}'.format(i))
             for i, size in enumerate(pooling_sizes))]))

        self.conv_sequence = ConvolutionalSequence(self.layers, num_channels,
                                                   image_size=image_shape)

        # Construct a top MLP
        self.top_mlp = MLP(top_mlp_activations, top_mlp_dims)

        # We need to flatten the output of the last convolutional layer.
        # This brick accepts a tensor of dimension (batch_size, ...) and
        # returns a matrix (batch_size, features)
        self.flattener = Flattener()
        application_methods = [self.conv_sequence.apply, self.flattener.apply,
                               self.top_mlp.apply]
        super(LeNet, self).__init__(application_methods, **kwargs)

    @property
    def output_dim(self):
        return self.top_mlp_dims[-1]

    @output_dim.setter
    def output_dim(self, value):
        self.top_mlp_dims[-1] = value

    def _push_allocation_config(self):
        self.conv_sequence._push_allocation_config()
        conv_out_dim = self.conv_sequence.get_dim('output')

        self.top_mlp.activations = self.top_mlp_activations
        self.top_mlp.dims = [numpy.prod(conv_out_dim)] + self.top_mlp_dims


def main(save_to, num_epochs, feature_maps=None, mlp_hiddens=None,
         conv_sizes=None, pool_sizes=None, batch_size=500,
         num_batches=None):
    if feature_maps is None:
        feature_maps = [20, 50]
    if mlp_hiddens is None:
        mlp_hiddens = [500]
    if conv_sizes is None:
        conv_sizes = [5, 5]
    if pool_sizes is None:
        pool_sizes = [2, 2]
    image_size = (28, 28)
    output_size = 10

    # Use ReLUs everywhere and softmax for the final prediction
    conv_activations = [Rectifier() for _ in feature_maps]
    mlp_activations = [Rectifier() for _ in mlp_hiddens] + [Softmax()]
    convnet = LeNet(conv_activations, 1, image_size,
                    filter_sizes=zip(conv_sizes, conv_sizes),
                    feature_maps=feature_maps,
                    pooling_sizes=zip(pool_sizes, pool_sizes),
                    top_mlp_activations=mlp_activations,
                    top_mlp_dims=mlp_hiddens + [output_size],
                    border_mode='full',
                    weights_init=Uniform(width=.2),
                    biases_init=Constant(0))
    # We push initialization config to set different initialization schemes
    # for convolutional layers.
    convnet.push_initialization_config()
    convnet.layers[0].weights_init = Uniform(width=.2)
    convnet.layers[1].weights_init = Uniform(width=.09)
    convnet.top_mlp.linear_transformations[0].weights_init = Uniform(width=.08)
    convnet.top_mlp.linear_transformations[1].weights_init = Uniform(width=.11)
    convnet.initialize()
    logging.info("Input dim: {} {} {}".format(
        *convnet.children[0].get_dim('input_')))
    for i, layer in enumerate(convnet.layers):
        logging.info("Layer {} ({}) dim: {} {} {}".format(
            i, layer.__class__.__name__, *layer.get_dim('output')))

    x = tensor.tensor4('features')
    y = tensor.lmatrix('targets')

    # Normalize input and apply the convnet
    probs = convnet.apply(x)
    cost = CategoricalCrossEntropy().apply(y.flatten(),
            probs).copy(name='cost')
    error_rate = MisclassificationRate().apply(y.flatten(), probs).copy(
            name='error_rate')

    cg = ComputationGraph([cost, error_rate])

    mnist_train = MNIST(("train",))
    mnist_train_stream = DataStream.default_stream(
        mnist_train, iteration_scheme=ShuffledScheme(
            mnist_train.num_examples, batch_size))

    mnist_test = MNIST(("test",))
    mnist_test_stream = DataStream.default_stream(
        mnist_test,
        iteration_scheme=ShuffledScheme(
            mnist_test.num_examples, batch_size))

    # Train with simple SGD
    algorithm = GradientDescent(
        cost=cost, parameters=cg.parameters,
        step_rule=Scale(learning_rate=0.1))
    # `Timing` extension reports time for reading data, aggregating a batch
    # and monitoring;
    # `ProgressBar` displays a nice progress bar during training.
    extensions = [Timing(),
                  FinishAfter(after_n_epochs=num_epochs,
                              after_n_batches=num_batches),
                  DataStreamMonitoring(
                      [cost, error_rate],
                      mnist_test_stream,
                      prefix="test"),
                  TrainingDataMonitoring(
                      [cost, error_rate,
                       aggregation.mean(algorithm.total_gradient_norm)],
                      prefix="train",
                      after_epoch=True),
                  Checkpoint(save_to),
                  ProgressBar(),
                  Printing()]

    model = Model(cost)

    main_loop = MainLoop(
        algorithm,
        mnist_train_stream,
        model=model,
        extensions=extensions)

    main_loop.run()

    
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    parser = ArgumentParser("An example of training a convolutional network "
                            "on the MNIST dataset.")
    parser.add_argument("--num-epochs", type=int, default=2,
                        help="Number of training epochs to do.")
    parser.add_argument("save_to", default="mnist.pkl", nargs="?",
                        help="Destination to save the state of the training "
                             "process.")
    parser.add_argument("--feature-maps", type=int, nargs='+',
                        default=[20, 50], help="List of feature maps numbers.")
    parser.add_argument("--mlp-hiddens", type=int, nargs='+', default=[500],
                        help="List of numbers of hidden units for the MLP.")
    parser.add_argument("--conv-sizes", type=int, nargs='+', default=[5, 5],
                        help="Convolutional kernels sizes. The kernels are "
                        "always square.")
    parser.add_argument("--pool-sizes", type=int, nargs='+', default=[2, 2],
                        help="Pooling sizes. The pooling windows are always "
                             "square. Should be the same length as "
                             "--conv-sizes.")
    parser.add_argument("--batch-size", type=int, default=500,
                        help="Batch size.")
    args = parser.parse_args()
    main(**vars(args))

usage: An example of training a convolutional network on the MNIST dataset.
       [-h] [--num-epochs NUM_EPOCHS]
       [--feature-maps FEATURE_MAPS [FEATURE_MAPS ...]]
       [--mlp-hiddens MLP_HIDDENS [MLP_HIDDENS ...]]
       [--conv-sizes CONV_SIZES [CONV_SIZES ...]]
       [--pool-sizes POOL_SIZES [POOL_SIZES ...]] [--batch-size BATCH_SIZE]
       [save_to]
An example of training a convolutional network on the MNIST dataset.: error: unrecognized arguments: -f


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [1]:
#RNN example from : http://blocks.readthedocs.org/en/latest/rnn.html

import numpy
import theano
from theano import tensor
from blocks import initialization
from blocks.bricks import Identity
from blocks.bricks.recurrent import SimpleRecurrent
x = tensor.tensor3('x')
rnn = SimpleRecurrent(
    dim=3, activation=Identity(), weights_init=initialization.Identity())
rnn.initialize()
h = rnn.apply(x)
f = theano.function([x], h)
print(f(numpy.ones((3, 1, 3), dtype=theano.config.floatX))) 

[[[ 1.  1.  1.]]

 [[ 2.  2.  2.]]

 [[ 3.  3.  3.]]]


In [ ]:

from blocks.algorithms.StepRule import StepRule
#http://blocks.readthedocs.org/en/latest/api/algorithms.html?highlight=scale#blocks.algorithms.StepRule
class MyOwnStepRule(StepRule)
    def __init__(self):
        pass
    
    def compute_step(parameter,previous_step):
        pass
    
    def compute_steps(previous_steps):
        pass

from blocks.bricks.conv import Convolutional

class HomeMadeConvNet(Feedfoward, Initializable):
    def __init__(self,filter_size=(5,5),num_channels=1):
        
        pass
    

class HomeMadeMaxoutNet(FeedforwardSequence, Initializable):
    def __init__(self, activations_layers):
        pass